# Question Answering su pagine web

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer()
loader = WebBaseLoader(
    web_paths=(
        "https://datamasters.it/corsi/",
        "https://datamasters.it/corsi/generative-ai/",
        "https://datamasters.it/corsi/machine-learning-per-tutti/",
        "https://datamasters.it/corsi/data-driven-mindset/",
        "https://datamasters.it/corsi/python-from-zero-to-hero/",
        "https://datamasters.it/corsi/ai-prompt-engineering/",
        "https://datamasters.it/corsi/corsi-data-scientist/",
        "https://datamasters.it/corsi/algotrading-in-python/",
        "https://corsi.datamasters.it/offers/8vpgpbtd/checkout",
        "https://datamasters.it/corsi/matematica-statistica-per-machine-learning/",
        "https://datamasters.it/corsi/machine-learning-starter-kit/",
        "https://datamasters.it/corsi/data-scientist-starter-kit/",
        "https://datamasters.it/corsi/machine-learning-foundations/"
    ),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [2]:
docs

[Document(metadata={'source': 'https://datamasters.it/corsi/', 'title': 'Catalogo dei Corsi sullâ€™AI - Data Masters', 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.", 'language': 'it-IT'}, page_content='\n\nCatalogo dei Corsi sullâ€™AI - Data Masters\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIMPARA A CREARE AGENTI AIÂ\xa0 Â\xa0PROMO 50% FINO AL 31 LUGLIO\nScopri di piÃ¹DataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChalle

In [3]:
len(docs[0].page_content)

12541

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=400, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [5]:
len(all_splits)

923

In [6]:
len(all_splits[42].page_content)

421

In [7]:
all_splits[42].metadata

{'source': 'https://datamasters.it/corsi/',
 'title': 'Catalogo dei Corsi sullâ€™AI - Data Masters',
 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.",
 'language': 'it-IT',
 'start_index': 10049}

In [8]:
all_splits[42]

Document(metadata={'source': 'https://datamasters.it/corsi/', 'title': 'Catalogo dei Corsi sullâ€™AI - Data Masters', 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.", 'language': 'it-IT', 'start_index': 10049}, page_content='Come posso contattare i docenti per eventuali domande o problemi?Puoi contattare direttamente i docenti del corso in qualsiasi momento per ricevere supporto e mentorship, nel canale dedicato al corso allâ€™interno della Community di Data Masters.\nPotrÃ² rivedere le lezioni anche dopo aver completato il corso/Percorso di Carriera?SÃ¬, avrai accesso per sempre a tutti i contenuti, inclusi eventuali aggiornamenti futuri.')

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6}) # similarity sta per cosine similarity

# metodi di ricerca alternativi presenti in ChromaDB (ma non in tutti i vector store):
# `similarity_score_threshold` - come similarity ma impostando una soglia minima di accettazione, scarta tutti i documenti sotto la soglia
# `mmr` = maximal marginal relevance - seleziona un insieme di esempi che sono sia simili all'input che diversi tra loro

In [11]:
retrieved_docs = retriever.invoke("consigliami un corso per iniziare la mia carriera da Data Scientist")

In [12]:
for doc in retrieved_docs:
    print(doc.metadata)

{'description': 'Diventa un Data Scientist con il Data Science Starter Kit, un percorso graduale from 0 to Hero studiato su misura per chi comincia.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/data-scientist-starter-kit/', 'start_index': 2, 'title': 'Corso online di Data Science e AI per chi parte da zero'}
{'description': 'Iscriviti alla Machine Learning Masterclass, il Percorso di Carriera in italiano più avanzato ed aggiornato per diventare un professionista un Machine Learning Engineer.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/corsi-data-scientist/', 'start_index': 23888, 'title': 'ML Engineer con la Machine Learning Masterclass | Data Masters'}
{'description': 'Diventa un Data Scientist con il Data Science Starter Kit, un percorso graduale from 0 to Hero studiato su misura per chi comincia.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/data-scientist-starter-kit/', 'start_index': 118, 'title': 'Corso online di Data Science 

In [13]:
retrieved_docs = retriever.invoke("vorrei diventare un Machine Learning Engineer")
for doc in retrieved_docs:
    print(doc.metadata)

{'description': 'Iscriviti alla Machine Learning Masterclass, il Percorso di Carriera in italiano più avanzato ed aggiornato per diventare un professionista un Machine Learning Engineer.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/corsi-data-scientist/', 'start_index': 674, 'title': 'ML Engineer con la Machine Learning Masterclass | Data Masters'}
{'description': 'Esplora il Machine Learning senza pre-requisiti tecnici: un corso che permette a tutti di avvicinarsi al mondo ML e AI', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/machine-learning-per-tutti/', 'start_index': 7703, 'title': 'Machine Learning per tutti | Corsi Data Masters'}
{'description': 'Iscriviti alla Machine Learning Masterclass, il Percorso di Carriera in italiano più avanzato ed aggiornato per diventare un professionista un Machine Learning Engineer.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/corsi-data-scientist/', 'start_index': 2024, 'title': 'ML Engineer con 

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt

C:\Cloud\GitHub\LLMs_MasterClass\.venv\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [16]:
example_messages = prompt.invoke(
    {"context": "contesto", "question": "domanda"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: domanda \nContext: contesto \nAnswer:", additional_kwargs={}, response_metadata={})]

In [17]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: domanda 
Context: contesto 
Answer:


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
rag_chain.invoke("mi piace l'intelligenza artificiale ma non saprei da dove iniziare a studiare, consigliami un corso")

'Ti consiglio di considerare il corso di Intelligenza Artificiale offerto da Data Masters, che è adatto anche ai principianti senza esperienza. Il corso inizia dalle basi e progredisce verso argomenti più avanzati, ed è disponibile online on-demand. Inoltre, i materiali didattici sono ben strutturati e facili da seguire.'

In [20]:
# recuperare i documenti selezionati per la generazione

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [21]:
risposta = rag_chain_with_source.invoke("come potrei approfondire tematiche di Deep Learning?")

risposta

{'context': [Document(metadata={'description': 'Iscriviti alla Machine Learning Masterclass, il Percorso di Carriera in italiano più avanzato ed aggiornato per diventare un professionista un Machine Learning Engineer.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/corsi-data-scientist/', 'start_index': 38424, 'title': 'ML Engineer con la Machine Learning Masterclass | Data Masters'}, page_content='le reti convolutive e ricorrenti, per poi esplorare il mondo del deep learning e delle reti neurali profonde, fino al reinforcement learning. Imparerai a progettare ed applicare algoritmi di machine learning su casi d’uso reali, guidato costantemente da professionisti che sperimentano quotidianamente le innovazioni del settore. Tantissime esercitazioni pratiche su Jupyter Notebook completano il percorso formativo, rendendovi capaci di operare sulle principali librerie oggi utilizzate, come TensorFlow, Keras, Scikit-Learn, Pandas, ecc.'),
  Document(metadata={'description': 'Is